# Title
- When you feel you need to use function in general, then there is a natural barier here and you should write it into script and import it

<!--- ![alt text](/home/patrik/pcflow/drawio/images/code.drawio.png) -->





# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from vis.deprecated_vis import *
import glob
from data.PATHS import DATA_PATH, TMP_VIS_PATH
from loss.flow import *

import argparse

# Structure Loss

# Use Case for NN Flow
Argoverse1 NSF - frame 2
<img src="/home/patrik/pcflow/use_cases/Smooth_NN_from_flow.png" width=600>

In [3]:
N1= 50
N2 = 55
K = 3
# na BS = 1
pc1 = torch.rand(1, N1, 3)
pc2 = torch.rand(1, N2, 3)
est_flow = torch.rand(1, N1, 3, requires_grad=True)

VOF, HOF = 100, 100 # from dataset
_, nn_ind, _ = knn_points(pc1, pc1, lengths1=None, lengths2=None, K=K, norm=1)

In [4]:
def chamfer_distance_loss(x, y, x_lengths=None, y_lengths=None, both_ways=False, use_normals=False, normals_K=5, loss_norm=1):
    '''
    Unique Nearest Neightboors?
    :param x:
    :param y:
    :param x_lengths:
    :param y_lengths:
    :param reduction:
    :return:
    '''
    if use_normals:
        normals1 = estimate_pointcloud_normals(x, neighborhood_size=normals_K)
        normals2 = estimate_pointcloud_normals(y, neighborhood_size=normals_K)

        x = torch.cat([x, normals1], dim=-1)
        y = torch.cat([y, normals2], dim=-1)


    x_nn = knn_points(x, y, lengths1=x_lengths, lengths2=y_lengths, K=1, norm=loss_norm)
    cham_x = x_nn.dists[..., 0]  # (N, P1)
    x_nearest_to_y = x_nn[1]

    if both_ways:
        y_nn = knn_points(y, x, lengths1=y_lengths, lengths2=x_lengths, K=1, norm=loss_norm)
        cham_y = y_nn.dists[..., 0]  # (N, P2)
        y_nearest_to_x = y_nn[1]

        nn_loss = (cham_x.mean() + cham_y.mean() ) / 2 # different shapes

    else:
        nn_loss = cham_x.mean()

    return nn_loss, cham_x, x_nearest_to_y

chamf_loss, per_point, x_to_y = chamfer_distance_loss(pc1 + est_flow, pc2, both_ways=False, use_normals=True, normals_K=K, loss_norm=1)

print(chamf_loss)

tensor(1.8669, grad_fn=<MeanBackward0>)


In [5]:
def smoothness_loss(est_flow, NN_idx, loss_norm=1, mask=None):

    bs, n, c = est_flow.shape

    if bs > 1:
        print("Smoothness Maybe not working, needs testing!")
    K = NN_idx.shape[2]

    est_flow_neigh = est_flow.view(bs * n, c)
    est_flow_neigh = est_flow_neigh[NN_idx.view(bs * n, K)]

    est_flow_neigh = est_flow_neigh[:, 1:K + 1, :]
    flow_diff = est_flow.view(bs * n, c) - est_flow_neigh.permute(1, 0, 2)

    flow_diff = (flow_diff).norm(p=loss_norm, dim=2)
    smooth_flow_loss = flow_diff.mean()
    smooth_flow_per_point = flow_diff.mean(dim=0).view(bs, n)

    return smooth_flow_loss, smooth_flow_per_point

# get normals
normals1 = estimate_pointcloud_normals(pc1, neighborhood_size=K)

pc_with_norms = torch.cat([pc1, normals1], dim=-1)
_, nn_ind_norms, _ = knn_points(pc_with_norms, pc_with_norms, lengths1=None, lengths2=None, K=K, norm=1)

smooth_loss, per_point = smoothness_loss(pc_with_norms, nn_ind_norms, loss_norm=1, mask=None)

print(smooth_loss)

tensor(1.0814)


# VIS

In [6]:
def mask_NN_by_dist(dist, nn_ind, max_radius):

    tmp_idx = nn_ind[:, :, 0].unsqueeze(2).repeat(1, 1, K).to(nn_ind.device)
    nn_ind[dist > max_radius] = tmp_idx[dist > max_radius]

    return nn_ind

vis_aware_nn_ind = strip_KNN_with_vis(pc1[0], nn_ind[0], VOF=VOF, HOF=HOF, margin=3)

# DT

In [7]:
import torch.nn.functional as F
import FastGeodis

class DT:
    def __init__(self, pc1, pc2, grid_factor=10):
        ''' works for batch size 1 only '''
        self.grid_factor = grid_factor
        pts = pc2[0]

        pc1_min = torch.min(pc1.squeeze(0), 0)[0]
        pc2_min = torch.min(pc2.squeeze(0), 0)[0]
        pc1_max = torch.max(pc1.squeeze(0), 0)[0]
        pc2_max = torch.max(pc2.squeeze(0), 0)[0]

        xmin_int, ymin_int, zmin_int = torch.floor(torch.where(pc1_min < pc2_min, pc1_min, pc2_min) * 10 - 1) / 10
        xmax_int, ymax_int, zmax_int = torch.ceil(torch.where(pc1_max > pc2_max, pc1_max, pc2_max) * 10 + 1) / 10

        pmin = (xmin_int, ymin_int, zmin_int)
        pmax = (xmax_int, ymax_int, zmax_int)

        sample_x = ((pmax[0] - pmin[0]) * grid_factor).ceil().int() + 2
        sample_y = ((pmax[1] - pmin[1]) * grid_factor).ceil().int() + 2
        sample_z = ((pmax[2] - pmin[2]) * grid_factor).ceil().int() + 2

        self.Vx = torch.linspace(0, sample_x, sample_x+1, device=pts.device)[:-1] / grid_factor + pmin[0]
        self.Vy = torch.linspace(0, sample_y, sample_y+1, device=pts.device)[:-1] / grid_factor + pmin[1]
        self.Vz = torch.linspace(0, sample_z, sample_z+1, device=pts.device)[:-1] / grid_factor + pmin[2]

        # NOTE: build a binary image first, with 0-value occuppied points
        grid_x, grid_y, grid_z = torch.meshgrid(self.Vx, self.Vy, self.Vz, indexing="ij")
        self.grid = torch.stack([grid_x.unsqueeze(-1), grid_y.unsqueeze(-1), grid_z.unsqueeze(-1)], -1).float().squeeze()
        H, W, D, _ = self.grid.size()
        pts_mask = torch.ones(H, W, D, device=pts.device)
        self.pts_sample_idx_x = ((pts[:,0:1] - self.Vx[0]) * self.grid_factor).round()
        self.pts_sample_idx_y = ((pts[:,1:2] - self.Vy[0]) * self.grid_factor).round()
        self.pts_sample_idx_z = ((pts[:,2:3] - self.Vz[0]) * self.grid_factor).round()
        pts_mask[self.pts_sample_idx_x.long(), self.pts_sample_idx_y.long(), self.pts_sample_idx_z.long()] = 0.

        iterations = 1
        image_pts = torch.zeros(H, W, D, device=pts.device).unsqueeze(0).unsqueeze(0)
        pts_mask = pts_mask.unsqueeze(0).unsqueeze(0)
        self.D = FastGeodis.generalised_geodesic3d(
            image_pts, pts_mask, [1./self.grid_factor, 1./self.grid_factor, 1./self.grid_factor], 1e10, 0.0, iterations
        ).squeeze()

    def torch_bilinear_distance(self, pc_deformed):

        pc_deformed = pc_deformed.squeeze(0)

        H, W, D = self.D.size()
        target = self.D[None, None, ...]

        sample_x = ((pc_deformed[:,0:1] - self.Vx[0]) * self.grid_factor).clip(0, H-1)
        sample_y = ((pc_deformed[:,1:2] - self.Vy[0]) * self.grid_factor).clip(0, W-1)
        sample_z = ((pc_deformed[:,2:3] - self.Vz[0]) * self.grid_factor).clip(0, D-1)

        sample = torch.cat([sample_x, sample_y, sample_z], -1)

        # NOTE: normalize samples to [-1, 1]
        sample = 2 * sample
        sample[...,0] = sample[...,0] / (H-1)
        sample[...,1] = sample[...,1] / (W-1)
        sample[...,2] = sample[...,2] / (D-1)
        sample = sample -1

        sample_ = torch.cat([sample[...,2:3], sample[...,1:2], sample[...,0:1]], -1)

        # NOTE: reshape to match 5D volumetric input
        dist = F.grid_sample(target, sample_.view(1,-1,1,1,3), mode="bilinear", align_corners=True).view(-1)


        return dist.mean(), dist


# NOTE: build DT map
dt = DT(pc1, pc2, grid_factor=10)
dt.torch_bilinear_distance(pc1[0])


(tensor(0.1526),
 tensor([0.1264, 0.1170, 0.0881, 0.0648, 0.1744, 0.2216, 0.2953, 0.2737, 0.1324,
         0.1848, 0.0982, 0.3610, 0.1031, 0.1185, 0.1607, 0.1485, 0.0874, 0.1753,
         0.2533, 0.2075, 0.1591, 0.1518, 0.0965, 0.1356, 0.1071, 0.1191, 0.1765,
         0.1496, 0.2589, 0.1643, 0.1347, 0.1229, 0.1099, 0.1183, 0.0884, 0.1432,
         0.1535, 0.1019, 0.1640, 0.1553, 0.1960, 0.0668, 0.1665, 0.1698, 0.1143,
         0.1272, 0.1737, 0.1524, 0.1758, 0.0824]))

# Forward flow
- forward flow with NN [x]
- smoothness from NN(pc2,pc2) to NN(pc1+flow, pc2)
- flow to same NN should be smooth or rigid kabsch


In [8]:
def forward_flow_loss(pc1, pc2, est_flow):
    ''' not yet for K > 1 or BS > 1
    Smooth flow on same NN from pc2 '''
    _, forward_nn, _ = knn_points(pc1 + est_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)

    a = est_flow[0] # magnitude

    ind = forward_nn[0] # more than one?

    #if pc1 is bigger than pc2, then skip?
    if pc1.shape[1] < pc2.shape[1]:
        shape_diff = pc2.shape[1] - ind.shape[0] + 1 # one for dummy    # what if pc1 is bigger than pc2?
        a = F.pad(a, (0,0,0, shape_diff), mode='constant', value=0)
        a.retain_grad() # padding does not retain grad, need to do it manually. Check it

        ind = F.pad(ind, (0,0,0, shape_diff), mode='constant', value=pc2.shape[1])  # pad with dummy not in orig

    # storage of same points
    vec = torch.zeros(ind.shape[0], 3)

    vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)  # will do forward flow

    # loss
    forward_loss = torch.nn.functional.mse_loss(vec[ind[:,0]], a, reduction='none').mean(dim=-1)

    return forward_loss.mean(), forward_loss[:est_flow.shape[1]]

f_loss, per_point_f_loss = forward_flow_loss(pc1, pc2, est_flow)

/tmp/ipykernel_1613/2860003869.py:21: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at /local/temporary/eb-build/PyTorch/1.13.0/foss-2022a-CUDA-11.7.0/pytorch-v1.13.0/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1615.)
  vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)  # will do forward flow


In [9]:
class UnsupervisedFlowLosses(torch.nn.Module):
    # update samples
    def __init__(self, args):
        super().__init__()
        self.args = args

        # store outputs of losses if better?
        self.current_loss = 10000

        # init losses
        self.loss_functions = []

        # [x] DT loss instead of chamfer
        # [x] ForwardFlow
        # todo push flow from freespace, mozna rikat visible-aware chamfer? []
        # solver uprava
        # kostka pro vyber NN / okoli


    def update(self, pc1, pc2):

        if args.loss_DT_weight > 0:
            self.dt = DT(pc1, pc2, grid_factor=10)
            # todo normals in DT?
        if args.loss_smooth_normals:
            normals1 = estimate_pointcloud_normals(pc1, neighborhood_size=args.loss_K_normals)
            pc_with_norms = torch.cat([pc1, normals1], dim=-1)

            # This is hence forward!
            pc1 = pc_with_norms

        if args.loss_smooth_weight > 0 and args.loss_vis_weight == 0:

            _, self.nn_ind, _ = knn_points(pc1, pc1, lengths1=None, lengths2=None, K=args.loss_KNN_smooth, norm=1)

        if args.loss_vis_weight > 0:
            dist, nn_ind, _ = knn_points(pc1, pc1, lengths1=None, lengths2=None, K=args.loss_KNN_vis, norm=1)
            mask_NN_by_dist(dist, nn_ind, args.loss_KNN_max_radius)
            # BS = 1
            self.nn_ind = strip_KNN_with_vis(pc1[0], nn_ind[0], VOF=VOF, HOF=HOF, margin=3).unsqueeze(0)


    def forward(self, pc1, pc2, est_flow):

        loss_dict = {}
        loss = 0

        if args.loss_smooth_weight > 0 or args.loss_vis_weight > 0:
            smooth_loss, smooth_per_point = smoothness_loss(est_flow, self.nn_ind, loss_norm=1, mask=None)

            loss_dict['smooth_loss'] = smooth_loss
            loss_dict['smooth_per_point'] = smooth_per_point

            loss += args.loss_smooth_weight * smooth_loss

        if args.loss_chamfer_weight > 0 and args.loss_DT_weight == 0:
            chamfer_loss, per_point_chamfer, x_to_y = chamfer_distance_loss(pc1 + est_flow, pc2, both_ways=args.loss_chamfer_both_ways, use_normals=args.loss_chamfer_use_normals, normals_K=args.loss_chamfer_normals_K, loss_norm=1)

            loss_dict['chamfer_loss'] = chamfer_loss
            loss_dict['chamfer_per_point'] = per_point_chamfer
            loss_dict['chamfer_x_to_y'] = x_to_y

            loss += args.loss_chamfer_weight * chamfer_loss

        if args.loss_DT_weight > 0:
            dt_loss, per_point_dt = self.dt.torch_bilinear_distance(pc1 + est_flow)

            loss_dict['dt_loss'] = dt_loss
            loss_dict['dt_per_point'] = per_point_dt

            loss += args.loss_DT_weight * dt_loss

        if args.loss_FF_weight > 0:
            FF_loss, per_point_FF = forward_flow_loss(pc1, pc2, est_flow)

            loss_dict['FF_loss'] = FF_loss
            loss_dict['FF_per_point'] = per_point_FF

            loss += args.loss_FF_weight * FF_loss

        loss_dict['loss'] = loss

        return loss_dict


args = argparse.Namespace()
args.loss_smooth_weight = 1.
args.loss_KNN_smooth = 3
args.loss_vis_weight = 0.
args.loss_smooth_normals = 1
args.loss_K_normals = 5
args.loss_KNN_vis = 3
args.loss_KNN_max_radius = 1.5

args.loss_chamfer_weight = 1.
args.loss_chamfer_both_ways = False
args.loss_chamfer_use_normals = False
args.loss_chamfer_normals_K = 5

args.loss_DT_weight = 0.
args.loss_DT_grid_factor = 10

args.loss_FF_weight = 1.
args.loss_FF_K = 1  # jine nema zatim

Loss = UnsupervisedFlowLosses(args)
Loss.update(pc1, pc2)
loss_dict = Loss.forward(pc1, pc2, est_flow)



In [10]:
# iterating over forward loss

init_flow = est_flow.detach().clone()
optimizer = torch.optim.Adam([est_flow], lr=0.1)

Loss_Function = UnsupervisedFlowLosses(args)
Loss_Function.update(pc1, pc2)

for it in range(300):

    loss_dict = Loss_Function(pc1, pc2, est_flow)

    loss = loss_dict['loss']
    loss.backward()

    print(loss)

    optimizer.step()
    optimizer.zero_grad()

print(loss_dict.keys())

tensor(2.0352, grad_fn=<AddBackward0>)
tensor(1.6313, grad_fn=<AddBackward0>)
tensor(1.3237, grad_fn=<AddBackward0>)
tensor(1.1034, grad_fn=<AddBackward0>)
tensor(0.9652, grad_fn=<AddBackward0>)
tensor(0.8808, grad_fn=<AddBackward0>)
tensor(0.8059, grad_fn=<AddBackward0>)
tensor(0.7510, grad_fn=<AddBackward0>)
tensor(0.7133, grad_fn=<AddBackward0>)
tensor(0.6693, grad_fn=<AddBackward0>)
tensor(0.6134, grad_fn=<AddBackward0>)
tensor(0.5664, grad_fn=<AddBackward0>)
tensor(0.5197, grad_fn=<AddBackward0>)
tensor(0.4953, grad_fn=<AddBackward0>)
tensor(0.4830, grad_fn=<AddBackward0>)
tensor(0.4887, grad_fn=<AddBackward0>)
tensor(0.4726, grad_fn=<AddBackward0>)
tensor(0.4544, grad_fn=<AddBackward0>)
tensor(0.4376, grad_fn=<AddBackward0>)
tensor(0.4334, grad_fn=<AddBackward0>)
tensor(0.4235, grad_fn=<AddBackward0>)
tensor(0.4091, grad_fn=<AddBackward0>)
tensor(0.3985, grad_fn=<AddBackward0>)
tensor(0.3833, grad_fn=<AddBackward0>)
tensor(0.3668, grad_fn=<AddBackward0>)
tensor(0.3603, grad_fn=<A

In [11]:
import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH
import matplotlib.cm as cm


color = cm.viridis(est_flow.norm(dim
                                 =-1).detach().numpy()[0])

plt.close()
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].plot(init_flow[0,:,0].detach(), init_flow[0,:,1].detach(), 'bo')
axes[0].plot(pc2[0,:,0].detach(), pc2[0,:,1].detach(), 'ro')

s = axes[1].scatter(est_flow[0,:,0].detach(), est_flow[0,:,1].detach(), c=per_point_f_loss.detach(), cmap=cm.viridis)

cb = plt.colorbar(mappable=s, ax=axes[1])
cb.set_label('Cbar Label Here')

# axes[1].grid(True, which='both', axis='x', linewidth=1)
plt.savefig(TMP_VIS_PATH + 'scatter.png')


In [12]:

# transfer args to pandas, to csv and back

import pandas as pd
args_list = [args, args]
values_list = []
names = [n[0] for n in args_list[0]._get_kwargs()]

for args in args_list:
    values = [n[1] for n in args._get_kwargs()]
    values_array = np.array((values))[None,:]
    values_list.append(values_array)

values_array = np.concatenate(values_list)

df = pd.DataFrame(values_array, columns=names, index=None)
df.to_csv('tst.csv')
# rewrite manually?

# Full loss example with animation
- write example points for use case [x]
- set up the losses for optimization of flow [x]
- store iterations in video []


In [12]:
# example points
pc1 = torch.tensor(((1.3,1,0), (1.6,1,0), (2.4, 1, 0))).unsqueeze(0)
pc2 = torch.tensor(((1.5,0.5,0), (1.8,0.6,0), (1.4, 0.2, 0), (1.6,0.2, 0), (1.5,0.2,0))).unsqueeze(0)
est_flow = torch.rand(pc1.shape, requires_grad=True)
est_flow.retain_grad()
# est_flow.requires_grad_()


In [4]:
plt.close()
plt.plot(pc1[0,:,0], pc1[0,:,1], 'bo')
plt.plot(pc2[0,:,0], pc2[0,:,1], 'ro')
plt.quiver(pc1[0,:,0], pc1[0,:,1], est_flow[0,:,0].detach(), est_flow[0,:,1].detach(), color='g')

plt.axis('equal')

plt.savefig(TMP_VIS_PATH + 'points.png')


In [13]:
# 1) FF loss
_, forward_nn, _ = knn_points(pc1 + est_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)

a = est_flow[0] # magnitude

ind = forward_nn[0] # more than one?

#if pc1 is bigger than pc2, then skip?
if pc1.shape[1] < pc2.shape[1]:
    shape_diff = pc2.shape[1] - ind.shape[0] + 1 # one for dummy    # what if pc1 is bigger than pc2?
    a = F.pad(a, (0,0,0, shape_diff), mode='constant', value=0)
    a.retain_grad() # padding does not retain grad, need to do it manually. Check it

    ind = F.pad(ind, (0,0,0, shape_diff), mode='constant', value=pc2.shape[1])  # pad with dummy not in orig

# storage of same points
vec = torch.zeros(ind.shape[0], 3)

vec = vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)  # will do forward flow

# print(ind)
# print(vec)

In [198]:
# 2) smoothness propagation
# K_SM = 3
# _, NN_pc2, _ = knn_points(pc2, pc2, lengths1=None, lengths2=None, K=K_SM, norm=1)
#
# keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]
#
# # print("\n\n Indices of pc2 for choosing only connections from NN(pc2) that has forward flow corespondences\n\n", keep_ind)
#
#
# # print("\n\n NN of pc2 that has correspondeces from flow\n\n", NN_pc2[0, keep_ind, :])
#
# # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
# n = NN_pc2[0, keep_ind, :]
#
# # beware of zeros
# connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)
# # print(connected_flow)
#
# # print(est_flow)
# prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=K_SM, dim=1)
#
# # smooth it, should be fine
# flow_comparison = prep_flow - connected_flow
#
# # print(flow_comparison.shape)
# # print(flow_comparison)
# per_point_loss = flow_comparison.norm(dim=2).mean(dim=1)




In [14]:
# optimize
K_SM = 3
optimizer = torch.optim.Adam([est_flow], lr=0.1)
_, NN_pc2, _ = knn_points(pc2, pc2, lengths1=None, lengths2=None, K=K_SM, norm=1)

for e in range(10):

    keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]

    # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
    n = NN_pc2[0, keep_ind, :]

    # beware of zeros
    connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)

    prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=K_SM, dim=1)

    # smooth it, should be fine
    flow_comparison = prep_flow - connected_flow

    per_point_loss = flow_comparison.norm(dim=2).mean(dim=1)
    loss = per_point_loss.mean()

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    print(loss)
# todo solve this issue


tensor(0.8229, grad_fn=<MeanBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.